In [1]:
import pandas as pd

In [2]:
def file_with_station(file,station):

    # Read file as text
    with open(file, "r", encoding="utf-8") as f:
        lines = f.readlines()

    # Find the row number where the data starts i.e. the row that contain 'date' in its first column
    header_row = None

    for i, line in enumerate(lines):
        # split on comma and strip spaces
        first_cell = line.split(",")[0].strip().lower()
        if first_cell == "date":
            header_row = i
            break
    
    # Read file as csv, delete uneccessary rows
    df = pd.read_csv(file, skiprows=header_row,low_memory=False)
    # modify the format of 'date'
    df['date'] = pd.to_datetime(df['date'], format='%d-%b-%Y %H:%M')
    # Add a column which takes the station name as value
    df["station"]= station
    # Add a column that contains only date details
    df['dateonly']= df['date'].dt.date
    # Add a column that contains only month-year
    df['yearmonth'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m')
    # Add a column that contains only month
    df['month'] = pd.to_datetime(df['date']).dt.strftime('%m')
    # Save as a new file in folder stationdata
    df.to_csv(f"stationdata/{station}.csv")
    print(f"The file {station}.csv is now created.")
    return

In [4]:
file_with_station("data/hly275.csv","MACE HEAD")
file_with_station("data/hly375.csv","OAK PARK")
file_with_station("data/hly518.csv","SHANNON AIRPORT")
file_with_station("data/hly532.csv","DUBLIN AIRPORT")
file_with_station("data/hly575.csv","MOORE PARK")
file_with_station("data/hly675.csv","BALLYHAISE")
file_with_station("data/hly775.csv","SHERKIN ISLAND")
file_with_station("data/hly875.csv","MULLINGAR")
file_with_station("data/hly1075.csv","ROCHES POINT")
file_with_station("data/hly1175.csv","NEWPORT")
file_with_station("data/hly1375.csv","DUNSANY")
file_with_station("data/hly1475.csv","GURTEEN")
file_with_station("data/hly1575.csv","MALIN HEAD")
file_with_station("data/hly1775.csv","JOHNSTOWN CASTLE 2")
file_with_station("data/hly1875.csv","ATHENRY")
file_with_station("data/hly1975.csv","MT DILLON")
file_with_station("data/hly2075.csv","FINNER")
file_with_station("data/hly2175.csv","CLAREMORRIS")
file_with_station("data/hly2275.csv","VALENTIA OBSERVATORY")
file_with_station("data/hly2375.csv","BELMULLET")
file_with_station("data/hly3723.csv","CASEMENT")
file_with_station("data/hly3904.csv","CORK AIRPORT")
file_with_station("data/hly4935.csv","KNOCK AIRPORT")

The file MACE HEAD.csv is now created.
The file OAK PARK.csv is now created.
The file SHANNON AIRPORT.csv is now created.
The file DUBLIN AIRPORT.csv is now created.
The file MOORE PARK.csv is now created.
The file BALLYHAISE.csv is now created.
The file SHERKIN ISLAND.csv is now created.
The file MULLINGAR.csv is now created.
The file ROCHES POINT.csv is now created.
The file NEWPORT.csv is now created.
The file DUNSANY.csv is now created.
The file GURTEEN.csv is now created.
The file MALIN HEAD.csv is now created.
The file JOHNSTOWN CASTLE 2.csv is now created.
The file ATHENRY.csv is now created.
The file MT DILLON.csv is now created.
The file FINNER.csv is now created.
The file CLAREMORRIS.csv is now created.
The file VALENTIA OBSERVATORY.csv is now created.
The file BELMULLET.csv is now created.
The file CASEMENT.csv is now created.
The file CORK AIRPORT.csv is now created.
The file KNOCK AIRPORT.csv is now created.


In [9]:
import sqlite3
from pathlib import Path

data_dir = "stationdata/"
db_path = "stations.db"

conn = sqlite3.connect(db_path)
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS station_dates (
    station TEXT,
    date DATE
)
""")

conn.commit()

In [ ]:
data_dir = Path("stationdata")

for file in data_dir.glob("*.csv"):
    # Read only 2 columns: 'station' and 'date'
    df = pd.read_csv(file, usecols=["station", "date"])
    # write to sqlite
    df.to_sql(
        "station_dates",
        conn,
        if_exists="append",
        index=False
    )

    print(f"  → loaded {len(df)} rows")


  → loaded 195792 rows
  → loaded 665435 rows
  → loaded 186264 rows
  → loaded 138192 rows
  → loaded 560280 rows
  → loaded 171504 rows
  → loaded 709296 rows
  → loaded 156408 rows
  → loaded 257154 rows
  → loaded 606658 rows
  → loaded 709297 rows
  → loaded 618768 rows
  → loaded 195528 rows
  → loaded 194184 rows
  → loaded 241627 rows
  → loaded 703464 rows
  → loaded 183360 rows
  → loaded 456408 rows
  → loaded 594311 rows
  → loaded 195792 rows
  → loaded 189240 rows
  → loaded 195504 rows
  → loaded 517340 rows


In [32]:
# change the type on 'wdsp' to numeric, the missing value will be turned into NaN with parameter errors="coerce"
df["wdsp"] = pd.to_numeric(df["wdsp"], errors="coerce")

# count the number of missing values
df["wdsp"].isna().sum()

np.int64(24)

In [33]:
# Set 'date' as index as interpolate based on actual time differences between index values
df = df.set_index('date')

# Interpolate missing windspeed (linear is best for meteorological data) -- I use AI to help me with this.
df['wdsp'] = df['wdsp'].interpolate(method='time', limit_direction='both', limit_area='inside')

# count the number of missing values
#df["wdsp"].isna().sum()

KeyError: "None of ['date'] are in the columns"